# **IEBS - Proyecto Final**

Este notebook se encarga a traves de las APi de twitter buscar los tweets de los diferentes bancos a ser evaluados. Con esto se genera un archivo csv de los tweets y de los usuarios. 

* Se incluyo la limpieza de los datos de text, para esto se incluyeron unas librerias nuevas que realizan este proceso. Se creo la funcion data_cleaning

Se generan 3 archivos:

* {id}.csv = Contine los datos de ese ID

* cleaning-

```
# Esto tiene formato de código
```

{id}.csv = Contiene los datos de ese ID con el text limpio 

* user-{id}.csv = Contiene los datos de los usuarios para los tweets buscados de ese id

# Instalacion de librerias

Instalacion de las librerias opcionales que son necesarias para la limpieza de los datos. Se ejecuta una sola vez en el notebook.

In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install datetime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52 kB 639 kB/s 
     |████████████████████████████████| 251 kB 10.5 MB/s 


In [ ]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 5.5 MB/s 


# Preparacion del entorno

Se prepara el entorno para ejecutar los tweets.

* se importan las librerias

* se conecta con el google drive para guardar los archivos

* se importan los datos para conectarse con twitter

In [97]:
# Importar el driver para almacenar los datos

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [98]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [99]:
# Se importan las librerias a utilizar durante todo el proyecto. 
import requests
from requests_oauthlib import OAuth1
import pandas as pd

from tweepy import OAuthHandler, API, Cursor
import json

import string
from nltk.tokenize import word_tokenize
from datetime import datetime
import unidecode
from unicodedata import normalize
import re
import warnings
warnings.filterwarnings('ignore')

In [100]:
# Se leen todos los datos para la conexion desde un archivo json
tweets_df = pd.read_json('/content/drive/MyDrive/datos/twitter.json')

# Se colocan las variables de trabajo
ckey = tweets_df['ckey'][0]
csecret = tweets_df['csecret'][0]
atoken = tweets_df['atoken'][0]
asecret = tweets_df['asecret'][0]
Bearer = tweets_df['Bearer'][0]

In [101]:
# Se genera una instancia de autenticacion
auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

In [102]:
# Se realiza una prueba del api. 
api = API(auth,
        wait_on_rate_limit=True,
        wait_on_rate_limit_notify=True)

# API con request

* Se invoca el API para traer los ID de los usuarios

* Se invoca el API para traer los 800 registros de cada ID

* Se realiza limpieza de los datos y se graban los csv correspondientes


In [103]:
# Se invoca el API que permite buscar a traves del nombre de usuario el ID para las busquedas de las menciones. 

headers = {'Authorization': f'Bearer {Bearer}'} # Variable de autorizador
url = f'https://api.twitter.com/2/users/by' # url de invocacion
bancos = '' # variable que guarda toda la respuesta

params = {'usernames': 'BBVA_espana,santander_es,caixabank,BancoSabadell,Bankinter',}

auth = OAuth1(ckey, csecret,
               atoken, asecret)

bancos = requests.get(url, params=params, headers=headers).json()
bancos

{'data': [{'id': '230123496',
   'name': 'BBVA en España',
   'username': 'BBVA_espana'},
  {'id': '1372470686', 'name': 'Santander España', 'username': 'santander_es'},
  {'id': '270429778', 'name': 'CaixaBank', 'username': 'caixabank'},
  {'id': '9980072', 'name': 'Banco Sabadell', 'username': 'BancoSabadell'},
  {'id': '86031609', 'name': 'Bankinter', 'username': 'Bankinter'}]}

In [104]:
# Se crea una funcion para buscar las menciones por ID, esta funcion devuelve dos DF, 
# El primero contiene todos los tweets que mencionan el ID del banco que corresponda. 
# El segundo contiene los usuarios que hacen mencion al ID. 

def buscar_tweets_mentions(mentions_id, cantidad_tweet=800):

    headers = {'Authorization': f'Bearer {Bearer}'} # Variable de autorizador
    url = f'https://api.twitter.com/2/users/{mentions_id}/mentions' # url de invocacion
    respuesta = '' # variable que guarda toda la respuesta
    usuarios = pd.DataFrame() # variable que guarda los usuarios
    data = pd.DataFrame()
    params = {
        'expansions': 'author_id,geo.place_id',
        'tweet.fields': 'conversation_id,created_at,lang,public_metrics,possibly_sensitive,source',
        'max_results':100} 

    auth = OAuth1(ckey, csecret,
               atoken, asecret)
    
    for cantidad in range(int(cantidad_tweet/100)):
        respuesta = requests.get(url, params=params, headers=headers).json()
        data = data.append(respuesta['data'])
        usuarios = usuarios.append(respuesta['includes']['users'])
        params = {
            'expansions': 'author_id,geo.place_id',
            'tweet.fields': 'conversation_id,created_at,lang,public_metrics,possibly_sensitive,source',
            'pagination_token': respuesta['meta']['next_token'],
            'max_results':100}
    return data, usuarios

# Data Cleaning

Este apartado realiza la limpieza de los datos de text.

## Dejar tweets con idioma español

In [115]:
def remove_idioma(df):
    return df.loc[df['lang'] == 'es', :]

## Eliminar menciones

In [116]:
def remove_link(text):
    text = " ".join(filter(lambda x:x[0:4]!='http', text.split()))
    return text

In [117]:
def remove_mencion(text):
    text = " ".join(filter(lambda x:x[0]!='@', text.split()))
    return text

## Se Eliminan los Caracteres Especiales

👇 Remove punctuation and lower case the text.

In [118]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [119]:
def remove_puntuation(text):
    mi_puntuacion = '!¡"#$%&\'()*+,-./:;<=>¿?@[\\]^_`{|}~'  
    for punctuation in mi_puntuacion:
        text = text.replace(punctuation, " ")
    return text

In [120]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

## Covierte en minusculas todo

In [121]:
def remove_lower(text1):
    text1 = text1.lower()
    return text1

## Elimina Acentos

In [122]:
def cambia_acentuadas(text):
    trans_tab = dict.fromkeys(map(ord, u'\u0301\u0308'), None)
    text = normalize('NFKC', normalize('NFKD', text).translate(trans_tab))
    return text

## Remueve Numeros

In [123]:
def remove_numbers(text1):
    text1 = ''.join(word for word in text1 if not word.isdigit())
    return text1

## Funcion de data_cleaning

In [124]:
def data_cleaning(df):

  df_cleaning = df

  # Eliminar elementos duplicados
#  df_cleaning.drop_duplicates(inplace=True)

  # Eliminar idiomas
  df_cleaning = remove_idioma(df_cleaning)

  # Eliminar menciones
  df_cleaning['text'] = df_cleaning['text'].apply(remove_mencion)

  # Eliminar los caracteres iniciales (espacio al principio y finales (espacion al final))
  df_cleaning['text'] = df_cleaning['text'].str.strip()

  # Elimina los caracteres especiales
  df_cleaning['text'] = df_cleaning['text'].apply(remove_puntuation)

  # Elimina los emoji
  df_cleaning['text'] = df_cleaning['text'].apply(remove_emoji)

  # Convierte todo en minusculas
  df_cleaning['text'] = df_cleaning['text'].apply(remove_lower)

  # Elimina acentos
  df_cleaning['text'] = df_cleaning['text'].apply(cambia_acentuadas)

  # Elimina numeros
  df_cleaning['text'] = df_cleaning['text'].apply(remove_numbers)

  return df_cleaning

# Ejecutar la busqueda 

In [125]:
#Se invoca la funcion pasando como dato el ID del usuario que previamente fue cargado. 

valores = bancos['data'] # Se toman los valores de la lista
for i in valores:
  id = i['id']
  # Se invoca la llamada a la funcion
  respuesta_funcion, usuarios_funcion = buscar_tweets_mentions(id)
  # Se ajusta la columna de fecha
  respuesta_funcion['created_at'] = pd.to_datetime(respuesta_funcion['created_at'])
  # Se guardan los dos archivos con el nombre del ID
  respuesta_funcion.to_csv(f'/content/drive/MyDrive/IEBS/Global Proyect/datos/{id}.csv')
  usuarios_funcion.to_csv(f'/content/drive/MyDrive/IEBS/Global Proyect/datos/user-{id}.csv')
  # Se realiza limpieza de variable
  respuesta_limpia = data_cleaning(respuesta_funcion)
  # Se guardan los dos archivos con el nombre del ID
  respuesta_limpia.to_csv(f'/content/drive/MyDrive/IEBS/Global Proyect/datos/cleaning-{id}.csv')
